In [6]:
import numpy as np

#원본 행렬 R 생성
R = np.array([[4, np.nan, np.nan, 2, np.nan],
              [np.nan, 5, np.nan, 3, 1],
              [np.nan, np.nan, 3, 4, 4],
              [5, 2, 1, 2, np.nan]])

num_users, num_items = R.shape

#잠재 요인 차원 지정
K=3

#P와 Q 행렬 초기화(정규 분포를 가진 임의의 값)
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [7]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)

    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]

    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [8]:
#R 행렬에서 0이 아닌 요소의 행 위치, 열 위치 저장
non_zeros = [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

#SGD 기법으로 P와 Q 행렬을 반복 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        eij = r - np.dot(P[i,:], Q[j,:].T)
        P[i,:] = P[i,:] + learning_rate * (eij * Q[j,:] - r_lambda * P[i,:])
        Q[j,:] = Q[j,:] + learning_rate * (eij * P[i,:] - r_lambda * Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)

    if (step%50) == 0:
        print('### iteration step:', step, ' rmse:', rmse)

### iteration step: 0  rmse: 3.2388050277987723
### iteration step: 50  rmse: 0.4876723101369648
### iteration step: 100  rmse: 0.1564340384819247
### iteration step: 150  rmse: 0.07455141311978046
### iteration step: 200  rmse: 0.04325226798579314
### iteration step: 250  rmse: 0.029248328780878973
### iteration step: 300  rmse: 0.022621116143829466
### iteration step: 350  rmse: 0.019493636196525135
### iteration step: 400  rmse: 0.018022719092132704
### iteration step: 450  rmse: 0.01731968595344266
### iteration step: 500  rmse: 0.016973657887570753
### iteration step: 550  rmse: 0.016796804595895633
### iteration step: 600  rmse: 0.01670132290188466
### iteration step: 650  rmse: 0.01664473691247669
### iteration step: 700  rmse: 0.016605910068210026
### iteration step: 750  rmse: 0.016574200475705
### iteration step: 800  rmse: 0.01654431582921597
### iteration step: 850  rmse: 0.01651375177473524
### iteration step: 900  rmse: 0.01648146573819501
### iteration step: 950  rmse: 0

In [9]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
